In [21]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="AzureProject")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-241963
Azure region: westeurope
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-241963


In [22]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "AzureProject"

# TODO: Create compute cluster
# Use vm_size = " Standard_D2_V2"in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# verify that cluser does not exist already
try:
    cpu_cluster=ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster= ComputeTarget.create(ws,cpu_cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# print detailed status for the current cluster
print(cpu_cluster.get_status().serialize())
    

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2023-09-28T13:43:21.534368+00:00', 'createdBy': {'userObjectId': '92a5563a-8782-4728-a447-f9eee0fbc083', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2023-09-28T13:48:25.942271+00:00', 'state': 'Running', 'vmSize': 'STANDARD_D2S_V3'}


In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{'--C':choice(0.01,0.1,1,10,50,100,500,1000),
'--max_iter':choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                     script='train.py',
                     compute_target=cpu_cluster,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=16,
                                    max_concurrent_runs=4)

In [26]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run= exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2695e6d6-fb22-4f99-b257-cd1444ecb878
Web View: https://ml.azure.com/runs/HD_2695e6d6-fb22-4f99-b257-cd1444ecb878?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-241963/workspaces/quick-starts-ws-241963&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-28T14:48:41.593024][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-09-28T14:48:42.1465911Z][SCHEDULER][INFO]Scheduling job, id='HD_2695e6d6-fb22-4f99-b257-cd1444ecb878_0' 
[2023-09-28T14:48:42.2930914Z][SCHEDULER][INFO]Scheduling job, id='HD_2695e6d6-fb22-4f99-b257-cd1444ecb878_1' 
[2023-09-28T14:48:42.3831550Z][SCHEDULER][INFO]Scheduling job, id='HD_2695e6d6-fb22-4f99-b257-cd1444ecb878_2' 
[2023-09-28T14:48:42.4609320Z][SCHEDULER][INFO]Scheduling job, id='HD_2695e6d6-fb22-4f99-b257-cd1444ecb878_3' 
[2023-09-28T14:48:42.419377][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error:     infer_column_types or _is_inference_required(set_column_types))\n  File \"/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/lib/python3.6/site-packages/azureml/data/dataset_factory.py\", line 1173, in _transform_and_validate\n    _validate_has_data(dataflow, 'Failed to validate the data.')\n  File \"/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/lib/python3.6/site-packages/azureml/data/dataset_error_handling.py\", line 68, in _validate_has_data\n    raise DatasetValidationError(error_message + '\\n' + e.compliant_message, exception=e)\nazureml.data.dataset_error_handling.DatasetValidationError: DatasetValidationError:\n\tMessage: Fail

In [ ]:
import joblib
# Get your best run and save the model from that run.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0,reverse =False, discard_no_metric=False))
### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metric:", best_run.get_metrics())
print("Run details",best_run.get_details() )
print("Best run file names", best_run.get_file_names())

In [ ]:
hypr_mdl='hyperdrive_model'
hyperdrive_model=best_run.register_model(model_name=hypr_mdl, model_path='.outputs/model.joblib')

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [ ]:
import os

os.getcwd()

In [ ]:
os.listdir()

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [12]:
# Submit your automl run
run=exp.submit(config=automl_config, show_output=True)


ConfigException: ConfigException:
	Message: Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "label_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        },
        "reference_code": "061ed905-e59e-42b9-ad95-cd18f40b4358"
    }
}

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model=run.get_output()
print(best_run)
print(fitted_model)

print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())